Дано: лог nginx,в архиве gz.

Надо: разобраться с форматом лога и преобразовать в формат csv. Т.е. чтобы параметры в строках были разделены точкой с запятой.

Зачем: это дает возможность анализировать лог в программах, которые могут читать csv (excel, библиотека pandas)

Подсказки: в python можно использовать
библиотеку gzip для чтения архива,
библиотеку re для обработки строк с помощью регулярных выражений

In [54]:
import gzip, re
import gzip

from ipywidgets import IntProgress
from IPython.display import display

import pandas as pd

Выводим первые 10 строк оригинального текста логов

In [55]:
inputFile = 'csv_example.csv.gz'
maxCnt = 3
with gzip.open(inputFile, 'rb') as f:
    cnt = 0
    for line in f:
        if (cnt == maxCnt):
            break
        line = line.decode().strip()
        #line = re.sub(r'(\w{3})\s+(\d+)\s+(\d{2}:\d{2}:\d{2})\s+(.+?)\s+(.+?)\s+(\d{2}\/\w{3}\/\d{4}:\d{2}:\d{2}:\d{2}\s+.+?)\s+(.+?)\s+-\s+(.+?)\s+(.+?)\s+(.+?)\s+"(.+?)"\s+(.+?)\s+(.+?)\s+(.+?)\s+"(.+?)"\s+"(.+?)"\s+-\s+(.+?\s+\d+.\d+)\s*',
        #              r'\1;\2;\3;\4;\5;\6;\7;\8;\9;\10;\11;\12;\13;\14;\15;\16;\17;',  
        #              line)
        print(line)
        cnt += 1

Aug  4 00:00:06 p00esianlbext01 esia.gosuslugi.ru_https_access: 03/Aug/2014:23:59:57 +0400 82.196.148.114 - GET /idp/resources/img/theme/sprite_arrow.png HTTP/1.1 "200" 0.001 1407095997.008 1434 "https://esia.gosuslugi.ru/idp/resources/css/theme.css" "Firefox/3.6.13, TestNVG" - upstream_responce: 0.001
Aug  4 00:00:06 p00esianlbext01 esia.gosuslugi.ru_https_access: 03/Aug/2014:23:59:57 +0400 109.165.86.241 - GET /idp/AuthnEngine HTTP/1.1 "302" 0.011 1407095997.009 311 "http://www.gosuslugi.ru/pgu/personcab" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36" - upstream_responce: 0.011
Aug  4 00:00:06 p00esianlbext01 esia.gosuslugi.ru_https_access: 03/Aug/2014:23:59:57 +0400 82.196.148.114 - GET /idp/resources/img/theme/primefaces/bg_paginator.png HTTP/1.1 "200" 0.002 1407095997.009 3498 "https://esia.gosuslugi.ru/idp/resources/css/theme.css" "Firefox/3.6.13, TestNVG" - upstream_responce: 0.002


Функция для определения количества повторений при обработке gzip файла построчно (используется в функции gzip_file_log_to_scv)

In [56]:
def getSizeFile(nameFile):
    size = 0

    with gzip.open(nameFile) as f:
      for line in f:
        size += 1
        pass
    return size

Функция для перевода файла с логами в (.gz) в csv с разделителем ;

Параметры:
- inputFile - имя исходного файла
- utputFile - имя результирующего файла
- regularForLog - регулярное выражение для 1 строки логов
- regularForOut - регулярное выражение на которое заменяется regularForLog
- maxCnt - количество строк из исходного файла, которые хотим перевести; по умолчанию -1 (т.е. весь файл)
- IntProgressMax - во время запуска функции появляется Progress bar, это параметр для него (а именно максимальное количество делений на которые он увеличивается) по умолчанию 100

In [57]:
def gzip_file_log_to_scv(inputFile, outputFile, regularForLog, regularForOut, maxCnt=-1, IntProgressMax=100):
    cntAddProgress = getSizeFile(inputFile) / IntProgressMax
    prgBar = IntProgress(min = 0, max = IntProgressMax) # Создаем прогрессбар
    display(prgBar) # Выводим прогрессбар на экран  
    
    with gzip.open(inputFile, 'rb') as f:
        with open(outputFile, 'w')as out:
            cnt = 0
            for line in f:
                if (cnt == maxCnt):
                    break #остановка для конкретного числа строчек
                elif (cnt % cntAddProgress == 0):
                    prgBar.value = prgBar.value + 1 #продвижение прогрессбара
                    
                line = line.decode().strip()
                line = re.sub(regularForLog, regularForOut, line)
                out.write(line + '\n')
                cnt += 1

Тестирование:
- добавление шапки для логов
- Перевод csv_example.csv.gz в csv

In [61]:
%%time

inputFile = 'csv_example.csv.gz'
outputFile = 'csv_example.csv'
headerNames = ['month', 'day', 'time', 
               'remote addr', 'remote user', 
               'time local','IP', 'Method', 
               'Path', 'Protocol', 'Status', 
               'Body bytes number1', 'number2', 'number3', 
              'http referer', 'http user agent', 'upstream_responce']
regularForLog = r'(\w{3})\s+(\d+)\s+(\d{2}:\d{2}:\d{2})\s+(.+?)\s+(.+?)\s+(\d{2}\/\w{3}\/\d{4}:\d{2}:\d{2}:\d{2}\s+.+?)\s+(.+?)\s+-\s+(.+?)\s+(.+?)\s+(.+?)\s+"(.+?)"\s+(.+?)\s+(.+?)\s+(.+?)\s+"(.+?)"\s+"(.+?)"\s+-\s+(.+?\s+\d+.\d+)\s*'                
regularForOut = r'"\1";"\2";"\3";"\4";"\5";"\6";"\7";"\8";"\9";"\10";"\11";"\12";"\13";"\14";"\15";"\16";"\17"'        

gzip_file_log_to_scv(inputFile, outputFile, regularForLog, regularForOut)

IntProgress(value=0)

Wall time: 2min 45s


Продолжение проверки:
- просмотр первых 5 строк с помощью бибилиотеки panda

In [62]:
df = pd.read_csv(outputFile, 
                 sep=';', 
                 error_bad_lines=False, 
                 low_memory=False, 
                 names=headerNames)

# Displays top 5 rows
df.head(5)

,month,day,time,remote addr,remote user,time local,IP,Method,Path,Protocol,Status,Body bytes sent,number1,number2,http referer,http user agent,upstream_responce
0,Aug,4,00:00:06,p00esianlbext01,esia.gosuslugi.ru_https_access:,03/Aug/2014:23:59:57 +0400,82.196.148.114,GET,/idp/resources/img/theme/sprite_arrow.png,HTTP/1.1,200,0.001,1407095997.008,1434,https://esia.gosuslugi.ru/idp/resources/css/th...,"Firefox/3.6.13, TestNVG",upstream_responce: 0.001
1,Aug,4,00:00:06,p00esianlbext01,esia.gosuslugi.ru_https_access:,03/Aug/2014:23:59:57 +0400,109.165.86.241,GET,/idp/AuthnEngine,HTTP/1.1,302,0.011,1407095997.009,311,http://www.gosuslugi.ru/pgu/personcab,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,upstream_responce: 0.011
2,Aug,4,00:00:06,p00esianlbext01,esia.gosuslugi.ru_https_access:,03/Aug/2014:23:59:57 +0400,82.196.148.114,GET,/idp/resources/img/theme/primefaces/bg_paginat...,HTTP/1.1,200,0.002,1407095997.009,3498,https://esia.gosuslugi.ru/idp/resources/css/th...,"Firefox/3.6.13, TestNVG",upstream_responce: 0.002
3,Aug,4,00:00:06,p00esianlbext01,esia.gosuslugi.ru_https_access:,03/Aug/2014:23:59:57 +0400,82.196.148.114,POST,/idp/staffUnitPaging,HTTP/1.1,200,0.005,1407095997.012,119,https://esia.gosuslugi.ru/idp/authn/UsernamePa...,"Firefox/3.6.13, TestNVG",upstream_responce: 0.005
4,Aug,4,00:00:06,p00esianlbext01,esia.gosuslugi.ru_https_access:,03/Aug/2014:23:59:57 +0400,95.55.0.200,POST,/profile/user/upStep3.xhtml,HTTP/1.1,200,0.097,1407095997.065,591,https://esia.gosuslugi.ru/profile/user/upStep3...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,upstream_responce: 0.097
